In [2]:
%pwd

'c:\\Users\\Dzuels Foundation\\mlproject\\notebook'

In [3]:
%cd ..\

c:\Users\Dzuels Foundation\mlproject


In [7]:

from src.constant import *
from src.logger import logging
from src.exception import CustomException
import requests
import os
import zipfile
import sys
from pathlib import Path
from dataclasses import dataclass
from src.utils import read_yaml,create_directory


In [14]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion Configuration
    """

    root_dir: Path
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path
    

In [15]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,):
        self.config = read_yaml(config_file_path)
                
        create_directory(self.config.artifact_root)
        
    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        
        create_directory(config.root_dir)
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config
        

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    
    def download_file(self):
        """
     Download a file from a URL and save it to the specified file path.
 
     Args:
         url (str): The URL to download the file from.
         file_path (str): The local path where the file will be saved.
 
     Returns:
         None
     """
        try:
            url = self.config.source_URL
            file_path = self.config.local_data_file
            response = requests.get(url)
            with open(file_path, 'wb') as f:
                f.write(response.content)
            logging.info(f"File downloaded successfully from {url} to {file_path}")
        except Exception as e:
            logging.info(f"Failed to download file from {url}. Error: {str(e)}")
            raise CustomException(e, sys)
        
    def extract_zip_file(self):
        """
        Extract a zip file to the specified directory.
        
        Args:
            file_path (str): The path of the zip file to unzip.
            extract_to (str): The directory to extract the contents to.
        """
        try:
            file_path = self.config.local_data_file
            extract_to = self.config.unzip_dir
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            logging.info(f"Unzipped '{file_path}' to '{extract_to}'.")
        except Exception as e:
            logging.info(f"Failed to unzip file from {file_path}. Error: {str(e)}")
            raise CustomException(e, sys)
    
        
            
         
         

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logging.info(f"An error occurred: {str(e)}")
    raise CustomException(e, sys)